In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.morphology import binary_dilation, dilation

In [2]:
def energize_neighbours(image, exploded=None, n_flash=0):
    """Energize neighbours in one epoch, one flash at a time"""
    selem = np.ones((3, 3))
    selem[1, 1] = 0 # Energize neighbours only, not itself
    if isinstance(exploded, type(None)):
        exploded = np.zeros_like(image, dtype=bool)
    newly_exploded = image > 9
    n_flash += np.sum(newly_exploded)
    inds = np.where(newly_exploded)
    exploded = np.logical_or(exploded, newly_exploded)
    image[exploded] = 0
    if len(inds[0]) > 0:
        for i, j in zip(inds[0], inds[1]): # We need to dilate around each flash individually
            mask = np.zeros_like(image)
            mask[i, j] = 1
            image += binary_dilation(mask, selem=selem).astype(int)
            image[exploded] = 0 # Just to make sure

    if np.sum(image > 9) > 0: # If new octopusses have reached high enough energy after neighbours had flashed
        # new flashing occurs -> neighbours get energized
        return energize_neighbours(image, exploded=exploded.astype(bool), n_flash=n_flash)

    return(image, n_flash)

In [3]:
# filename = "./data/day11test.txt"
filename = "./data/day11a.txt"
with open(filename, 'r') as ff:
    data = ff.read()

puss = np.array([[int(x) for x in line] for line in data.split("\n")])

In [4]:
counter = 0
n_flashes = [0] # no flashes in initial configuration
while True:
    puss+=1
    counter+=1 # epoch counter
    slika, broj = energize_neighbours(puss)
    n_flashes.append(broj) # contains number of flashes for each epoch
    if np.all(slika == slika[0,0]): # if all equal
        break

## part 1

In [5]:
# Accumulated total of flashes in epoch 100
np.cumsum(n_flashes)[100]

1667

## part 2

In [6]:
# When did the octopuses first synchronized?
counter

488

## Animation

In [7]:
# %load "./visualisations/scripts/day11_dumbo_animation.py"

![Dambo Animation](./visualisations/day11a_Dambos.gif)